In [5]:
from sklearn.datasets import make_classification
from train_model import train_network
import torch
import numpy as np
from matplotlib import pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [2]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
%matplotlib inline
from torch.utils.data import Dataset, DataLoader

In [3]:
n_mosaic = 3000
K = 10
d = 4
n_classes = 5

In [4]:
x,y = make_classification(n_samples=500,n_features=d,n_informative=d,n_redundant=0,n_repeated= 0,n_classes=n_classes,n_clusters_per_class=1,flip_y=0.20,random_state=1234)   #D_4 random_state 1234 flipy 0.20    D_5 1236 flipy 0.15


In [5]:
data = {'X':x,'Y':y}
np.save("data_2.npy",data)

In [6]:
# fg vs bg
y1 = np.zeros(y.shape,dtype=np.long)
indices = np.logical_or(y == 0,y==1)   #  0 and 1 indices are fg 
y1[indices] = 0   #fg indices at index 0
y1[np.logical_not(indices)] = 1  #bg index at index1

# fg1 vs fg2
index = np.logical_or(y==0,y==1)
x2 = x[index,:]
y2 = np.zeros(index.shape,dtype=np.long)
y2 = y[index]

In [7]:
class MosaicDataset(Dataset):
  """MosaicDataset dataset."""

  def __init__(self, mosaic_list, mosaic_label):
    """
      Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.mosaic = mosaic_list
    self.label = mosaic_label
    

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    return self.mosaic[idx] , self.label[idx]

batch = 250
# dataset = MosaicDataset(x2,y2)
# traindataloader = DataLoader( dataset,batch_size= batch ,shuffle=True)

# linear Models (fgvsbg and fg1vsfg2)

In [ ]:
class Net_linear(nn.Module):
  def __init__(self):
    super(Net_linear,self).__init__()
    self.linear1 = nn.Linear(d,2)
    
  def forward(self,x):
    x = self.linear1(x)
   
    return x

In [6]:
dataset = MosaicDataset(x,y1)
traindataloader = DataLoader( dataset,batch_size= batch ,shuffle=True)
net = Net_linear().double()
net =net.to(device)
train_model =train_network(net,traindataloader)
train_model.training(epochs=200,mini=1)



In [7]:
_,_=train_model.predict(traindataloader,True)
plt.plot(train_model.train_loss)

***Saving Pretrained Focus Network fg vs bg***

In [ ]:
train_model.save_models("pretrained_focusl_net")

In [8]:
dataset = MosaicDataset(x2,y2)
traindataloader = DataLoader( dataset,batch_size= batch ,shuffle=True)
net = Net_linear().double()
net =net.to(device)
train_model =train_network(net,traindataloader)
train_model.training(epochs=100,mini=1)



In [9]:
_,_=train_model.predict(traindataloader,True)
plt.plot(train_model.train_loss)

***Saving Classification Network fg1 vs fg2***

In [ ]:
train_model.save_models("pretrained_classifyl_net")

# train deep models (fgvsbg and fg1vsfg2)

In [ ]:
class Net_deep(nn.Module):
  def __init__(self):
    super(Net_deep,self).__init__()
    self.linear1 = nn.Linear(d,50)
    self.linear2 = nn.Linear(50,2)
  def forward(self,x):
    x = F.relu(self.linear1(x))
    x = self.linear2(x)
    
    return x

In [10]:
dataset = MosaicDataset(x,y1)
traindataloader = DataLoader( dataset,batch_size= batch ,shuffle=True)
net = Net_deep().double()
net =net.to(device)
train_model =train_network(net,traindataloader)
train_model.training(epochs=200,mini=1)

In [11]:
_,_=train_model.predict(traindataloader,True)
plt.plot(train_model.train_loss)

***Saving Pretrained Focus Network fg vs bg***

In [ ]:
train_model.save_models("pretrained_focusd_net")

In [12]:
dataset = MosaicDataset(x2,y2)
traindataloader = DataLoader( dataset,batch_size= batch ,shuffle=True)
net = Net_deep().double()
net =net.to(device)
train_model =train_network(net,traindataloader)
train_model.training(epochs=100,mini=1)


In [13]:
_,_=train_model.predict(traindataloader,True)
plt.plot(train_model.train_loss)

***Saving Classification Network fg1 vs fg2***

In [ ]:
train_model.save_models("pretrained_classifyd_net")

# Mosaic Data

In [14]:
from plots import plot_analysis,focus_map,classification_map
from Models_Elemental import Focus_linear,Classification_linear
from Models_Elemental import Focus_deep,Classification_deep
from train_mosaic_new import train_mosaic_network

In [15]:
idx= []
for i in range(n_classes):
    print(i,sum(y==i))
    idx.append(y==i)

In [10]:
foreground_classes = {'class_0' ,'class_1'}

background_classes = {'class_2','class_3'}#,'class_4', 'class_5', 'class_6','class_7', 'class_8', 'class_9'}

In [11]:

mosaic_list =[]
mosaic_label = []
fore_idx=[]
for j in range(n_mosaic):
    fg_class  = np.random.randint(0,len(foreground_classes))
    fg_idx = np.random.randint(0,K)
    a = []
    for i in range(K):
        if i == fg_idx:
            b = np.random.choice(np.where(idx[fg_class]==True)[0],size=1)
            a.append(x[b])
#             print("foreground "+str(fg_class)+" present at " + str(fg_idx))
        else:
            bg_class = np.random.randint(len(foreground_classes),n_classes)
            b = np.random.choice(np.where(idx[bg_class]==True)[0],size=1)
            a.append(x[b])
#             print("background "+str(bg_class)+" present at " + str(i))
    a = np.concatenate(a,axis=0)
    mosaic_list.append(np.reshape(a,(d*K,1)))
    mosaic_label.append(fg_class)
    fore_idx.append(fg_idx)

In [16]:
mosaic_list = np.concatenate(mosaic_list,axis=1).T
mosaic_list.shape

In [17]:
np.unique(mosaic_label)

In [14]:
mosaic_data = {"X":mosaic_list,"Y":mosaic_label,"foreground_indices":fore_idx}
np.save("mosaic_data_4.npy",mosaic_data)

# Linear Support Vector Classifier on Mosaic Data

In [18]:
from sklearn.svm import SVC
sv = SVC(C=100,kernel="linear")
sv.fit(mosaic_list,mosaic_label)
print(sv.score(mosaic_list,mosaic_label))

# Deep Network For Mosaic Data

In [15]:
class Net1(nn.Module):
  def __init__(self):
    super(Net1,self).__init__()
    self.linear1 = nn.Linear(d*K,50)
    self.linear2 = nn.Linear(50,50)
    self.linear3 = nn.Linear(50,50)
    self.linear4 = nn.Linear(50,2)
  def forward(self,x):
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = self.linear4(x)
    return x

In [16]:
class MosaicData(Dataset):
  """MosaicDataset dataset (without foreidx)."""

  def __init__(self, mosaic_list, mosaic_label):
    """
      Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.mosaic = mosaic_list
    self.label = mosaic_label

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    return self.mosaic[idx] , self.label[idx]

batch = 250

In [19]:
dataset = MosaicData(mosaic_list,mosaic_label)
mosaicdataloader = DataLoader( dataset,batch_size= batch ,shuffle=True)
net1 = Net1().double()
net1 =net1.to(device)
train_mosaic =train_network(net1,mosaicdataloader)
train_mosaic.training(epochs=200,mini=1)

In [20]:
_,_=train_mosaic.predict(mosaicdataloader,True)
plt.plot(train_mosaic.train_loss)

# Attention Networks

In [15]:
class MosaicDataset(Dataset):
  """MosaicDataset dataset."""

  def __init__(self, mosaic_list, mosaic_label, fore_idx):
    """
      Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.mosaic = mosaic_list
    self.label = mosaic_label
    self.fore_idx = fore_idx

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    return self.mosaic[idx] , self.label[idx], self.fore_idx[idx]

batch = 250
msd = MosaicDataset(mosaic_list, mosaic_label , fore_idx)
train_loader = DataLoader( msd,batch_size= batch ,shuffle=True)

# linear Attention network

In [ ]:
pretrained_classify = True
pretrained_focus = True
if pretrained_classify == True:
  where = Focus_linear(d,1,K,d).double()
  what = Classification_linear(d,2).double()
  what.load_state_dict( torch.load("pretrained_classifyl_net.pt"))

if pretrained_focus == True:
  where = Focus_linear(d,2,K,d,pretrained=True).double()
  what = Classification_linear(d,2).double()
  where.load_state_dict( torch.load("pretrained_focusl_net.pt"))
if pretrained_focus == False and pretrained_classify == False:
    where = Focus_linear(d,1,K,d).double()
    what = Classification_linear(d,2).double()
elif pretrained_focus ==True and pretrained_classify ==True:
  where = Focus_linear(d,2,K,d,pretrained=True).double()
  what = Classification_linear(d,2).double()
  what.load_state_dict( torch.load("pretrained_classifyl_net.pt"))
  where.load_state_dict( torch.load("pretrained_focusl_net.pt"))



In [ ]:
train_mosaic = train_mosaic_network(where,what,train_loader,elemental=False,lr=0.01)

In [21]:
train_mosaic.training(epochs=200,mini=3,train_focus=True,train_classify=False)

In [22]:
plot_analysis(np.array(train_mosaic.train_analysis))

In [23]:
plt.plot(train_mosaic.train_loss)

In [24]:
_,_,_,_ = train_mosaic.predict(train_loader,True)

#  deep Attention Networks

In [ ]:
pretrained_classify = True
pretrained_focus = False
if pretrained_classify == True:
  where_deep = Focus_deep(d,1,K,d).double()
  what_deep = Classification_deep(d,2).double()
  what_deep.load_state_dict( torch.load("pretrained_classifyd_net.pt"))

if pretrained_focus == True:
  where_deep = Focus_deep(d,2,K,d,pretrained=True).double()
  what_deep = Classification_deep(d,2).double()
  where_deep.load_state_dict( torch.load("pretrained_focusd_net.pt"))
if pretrained_focus == False and pretrained_classify == False:
  where_deep = Focus_deep(d,1,K,d).double()
  what_deep = Classification_deep(d,2).double()
elif pretrained_focus ==True and pretrained_classify ==True:
  where_deep = Focus_deep(d,2,K,d,pretrained=True).double()
  what_deep = Classification_deep(d,2).double()
  what_deep.load_state_dict( torch.load("pretrained_classifyd_net.pt"))
  where_deep.load_state_dict( torch.load("pretrained_focusd_net.pt"))



In [25]:
train_mosaic_deep = train_mosaic_network(where_deep,what_deep,train_loader,elemental=False,lr =0.01)
train_mosaic_deep.training(epochs=700,mini=3,train_focus=False,train_classify=True)

In [28]:
plot_analysis(np.array(train_mosaic_deep.train_analysis))

In [4]:
plt.plot(train_mosaic_deep.train_loss)

In [3]:
_,_,_,_ = train_mosaic_deep.predict(train_loader,True)

## linear deep attention network

In [50]:
pretrained_classify = True
pretrained_focus = True
if pretrained_classify == True:
  where_deep = Focus_linear(d,1,K,d).double()
  what_deep = Classification_deep(d,2).double()
  what_deep.load_state_dict( torch.load("pretrained_classifyd_net.pt"))

if pretrained_focus == True:
  where_deep = Focus_linear(d,2,K,d,pretrained=True).double()
  what_deep = Classification_deep(d,2).double()
  where_deep.load_state_dict( torch.load("pretrained_focusl_net.pt"))
if pretrained_focus == False and pretrained_classify == False:
  where_deep = Focus_linear(d,1,K,d).double()
  what_deep = Classification_deep(d,2).double()
elif pretrained_focus ==True and pretrained_classify ==True:
  where_deep = Focus_linear(d,2,K,d,pretrained=True).double()
  what_deep = Classification_deep(d,2).double()
  what_deep.load_state_dict( torch.load("pretrained_classifyd_net.pt"))
  where_deep.load_state_dict( torch.load("pretrained_focusl_net.pt"))

In [2]:
train_mosaic_deep = train_mosaic_network(where_deep,what_deep,train_loader,elemental=False,lr =0.01)
train_mosaic_deep.training(epochs=700,mini=3,train_focus=False,train_classify=True)
plot_analysis(np.array(train_mosaic_deep.train_analysis))
#plt.plot(train_mosaic_deep.train_loss)
_,_,_,_ = train_mosaic_deep.predict(train_loader,True)

# deep linear attention network

In [86]:
pretrained_classify = True
pretrained_focus = True
if pretrained_classify == True:
  where_deep = Focus_deep(d,1,K,d).double()
  what_deep = Classification_linear(d,2).double()
  what_deep.load_state_dict( torch.load("pretrained_classifyl_net.pt"))

if pretrained_focus == True:
  where_deep = Focus_deep(d,2,K,d,pretrained=True).double()
  what_deep = Classification_linear(d,2).double()
  where_deep.load_state_dict( torch.load("pretrained_focusd_net.pt"))
if pretrained_focus == False and pretrained_classify == False:
  where_deep = Focus_deep(d,1,K,d).double()
  what_deep = Classification_linear(d,2).double()
elif pretrained_focus ==True and pretrained_classify ==True:
  where_deep = Focus_deep(d,2,K,d,pretrained=True).double()
  what_deep = Classification_linear(d,2).double()
  what_deep.load_state_dict( torch.load("pretrained_classifyl_net.pt"))
  where_deep.load_state_dict( torch.load("pretrained_focusd_net.pt"))

In [1]:
train_mosaic_deep = train_mosaic_network(where_deep,what_deep,train_loader,elemental=False,lr =0.01)
train_mosaic_deep.training(epochs=700,mini=3,train_focus=False,train_classify=True)
plot_analysis(np.array(train_mosaic_deep.train_analysis))
#plt.plot(train_mosaic_deep.train_loss)
_,_,_,_ = train_mosaic_deep.predict(train_loader,True)